-This note book is the iterartion 05 of the HPCC_GNN_Project-
+ we were asked to use the function NPList2Tens which needed the nparray as its parameter along with the correct shape ,
  this is just the modified verison of the '#01_GNN_Project_backup'
+ To obtain the size of the nparray we are utilizing the np.expanddims 

In [1]:
import io
from PIL import Image
import binascii

In [2]:
# img=Image.open("C:/Users/rohan/Pictures/Camera Roll/X13_wallpaper_final_16x9_FHD.jpg")
img=Image.open("C:/Users/rohan/Pictures/Saved Pictures/img01.png")
print(img)


<PIL.PngImagePlugin.PngImageFile image mode=P size=2000x1200 at 0x21B894B26E0>


Here the image must be opened in read bytes format

In [6]:
# with open("C:/Users/rohan/Pictures/Camera Roll/X13_wallpaper_final_16x9_FHD.jpg", "rb") as image_file:
with open("C:/Users/rohan/Pictures/Saved Pictures/img01.png", "rb") as image_file:
    # image_hex = binascii.hexlify(image_file.read())
    print(len(image_file.read()))
    # print(image_he
    # print(image_hex[:1500])
    

1279102


the hex data into binary data using unhexlify() function.

In [4]:
import binascii
import pickle
binary_data = binascii.unhexlify(image_hex)
# header_index = binary_data.find(b'\xff\xd8') 
# stripped_image_bytes = binary_data[header_index:]
# stripped_image_hex = binascii.hexlify(stripped_image_bytes)
# print(stripped_image_hex)

# for i in range(0,15):
#     print( binary_data[i])
# print(header_index)
# with open('C:/Users/rohan/Desktop/uktkjxhcb/HPCC_GNN project backup/pngBinaryDataDump.txt', 'wb') as f:
#     pickle.dump(binary_data, f)
# f.close();


To check if the image data is png we check the first 16 bits

In [5]:
png_signature = "89504e470d0a1a0a"
if image_hex[:16] == png_signature:
    image_hex = image_hex[16:]

In [6]:
# import numpy as np

# # Convert the bytes to a NumPy array 
# # Interpret a buffer as a 1-dimensional array.


# stripped_image_array = np.frombuffer(binary_data, dtype=np.uint8)
# np.expand_dims(stripped_image_array,1)
# print(stripped_image_array)


The method strip_headers(binary_data) using the Pillow library strips the headers from the PNG image.
The Image.open() method reads the binary data as an image and automatically strips the headers and metadata to give you a representation of the image data.
The image data is then converted to a NumPy array using np.array(). 
This gives you a NumPy array that only contains the image pixel data, with the headers and metadata stripped.

In [7]:
from PIL import Image
from io import BytesIO
import numpy as np

def strip_headers(binary_data):
    with Image.open(BytesIO(binary_data)) as im:
        return np.array(im)

stripped_image_array1 = strip_headers(binary_data)
print(stripped_image_array1)


[[ 7  7  7 ...  2  2  2]
 [ 7  7  7 ...  2 22 22]
 [ 7  7  7 ...  2 22 22]
 ...
 [ 5 18  5 ... 71 71 71]
 [ 5  5 18 ... 71 71 71]
 [ 5  4 18 ... 71 71 71]]


function to interpret the binary data as an image and return the image in the form of a tensor.

In [8]:
#use numpyyy
#'image' library to strip the headers
#reshape by looking into the header
import tensorflow as tf
image_tensor =tf.convert_to_tensor(stripped_image_array1)
print(image_tensor)


tf.Tensor(
[[ 7  7  7 ...  2  2  2]
 [ 7  7  7 ...  2 22 22]
 [ 7  7  7 ...  2 22 22]
 ...
 [ 5 18  5 ... 71 71 71]
 [ 5  5 18 ... 71 71 71]
 [ 5  4 18 ... 71 71 71]], shape=(1200, 2000), dtype=uint8)


As it is clear that the returned tensor is having a wrong dimensions and thus we can use the np.expanddims()

In [11]:
image_array_with_batch_dim = np.expand_dims(stripped_image_array1, axis=0)
print(image_array_with_batch_dim.shape)
image_tensor_added_dims =tf.convert_to_tensor(image_array_with_batch_dim)
print(image_tensor_added_dims)


(1, 1200, 2000)
tf.Tensor(
[[[ 7  7  7 ...  2  2  2]
  [ 7  7  7 ...  2 22 22]
  [ 7  7  7 ...  2 22 22]
  ...
  [ 5 18  5 ... 71 71 71]
  [ 5  5 18 ... 71 71 71]
  [ 5  4 18 ... 71 71 71]]], shape=(1, 1200, 2000), dtype=uint8)


REVERTING THE TENSOR BACK TO THE IMAGE

In [9]:
np_ar = image_tensor.numpy()
print(np_ar)



[[ 7  7  7 ...  2  2  2]
 [ 7  7  7 ...  2 22 22]
 [ 7  7  7 ...  2 22 22]
 ...
 [ 5 18  5 ... 71 71 71]
 [ 5  5 18 ... 71 71 71]
 [ 5  4 18 ... 71 71 71]]


In [ ]:
# nd arrray to bytes
bts = np_ar.tobytes()
print(bts)

In [11]:
# bytes to hex
res = binascii.hexlify(bytearray(bts))
print(res[:1900])

b'0707070707070707070707071907070707070707070707070707070707070707070707191919190707070719070719191919191919190f191919191919190f0f0f0f0f1b0f0f0f1b0f0f0f0f0f0f0f0f0f0f1b1b1b1b1b1b0a1b0a0a0a0a0a0a0a0a1b0a0a0a0a0a0a0a0a0a0a0a0a0a0a0a0a0a0a0a16161616161616161616161616161616161616161616161616161602020216020202020202020e0202020202020e0e0e0e0e0e0e0e0e0e0e02020e0e0e0e0e1a0e02020e0e0e1a1a1a1a1a1a1a1a1a1a1a1a1313131313131313131355552e2e2e2e2e2e2e2e2c2c2c55131e4545453f453939454545453939452c2e1e453f4b6b21582f5821362121212121216b4444452c2c1e1e2c552e3f4545451e551a0e0e1a1a0e025545453f1436584414144b214b44144b143f1e552c3f550e020e0e1a1a0e1a132e131a0e02160e1a1a1a2e45144444144426266b6b4b215821146b36216b4b4b362121212636365858263621212126582f5836212158582636585826212f58216b213636363921582144212f4b36266b4b21216b216b4b44452c1a02020e1a131a0e132e552e1e3f2e0e1a2e1e1e130e0e1a2c1a160a16161616021a1e45453f4539451e2e45143f2e2e2e552c3921143f1e3f2e0e0e2e2c1e1e2e551e451439393945454545453f4539393f3f3945453f3f39453939452c

In [12]:
import tensorflow as tf

# Create a tensor
tensor = image_tensor

# Convert the tensor to a string
tensor_str = str(tensor.numpy().tolist())

# Write the string to a file
with open('C:/Users/rohan/Desktop/uktkjxhcb/HPCC_GNN project backup/pngImgTensorDump.txt', 'w') as f:
    f.write(tensor_str)
    f.close()

In [13]:
import pickle

serialized_image_tensor = pickle.dumps(image_tensor)

with open('C:/Users/rohan/Desktop/uktkjxhcb/HPCC_GNN project backup/pngImgTensorDump.txt', 'wb') as f:
    f.write(serialized_image_tensor)



